In [ ]:
import pandas as pd
import datetime 
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
SQVI_BLOQUEI = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\SQVI_BLOQUEI_BLOQUEI.xlsx')
dados = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\ZCO144.xlsx')
Estoque_Sql = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\estoque.xlsx')
Padrão_de_material = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\Padrão de material.xlsx')
ULTIMA_VENDA = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\ULTIMA_VENDA.xlsx')
ME2M = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\ME2M.xlsx')
sqvi_zpp89_frequencia_venda = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\sqvi_zpp89_frequencia_venda_frequencia_venda.xlsx')
ZSD138_CARTE = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\ZSD138_CARTE.xlsx')
ZPP101 = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\ZPP101.xlsx')
ZMM34_VENDA = pd.read_excel('U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Saldos\\ZMM34_VENDA.xlsx')

In [ ]:
SQVI_BLOQUEI.head()

In [ ]:
#Drop #eliminar coluna unidade "Un"
todos_fert_sql = SQVI_BLOQUEI[['Material']]
#lista de materiais zco144  
fert_zco144 = dados[['Material']]

#verificando juntação lista de materiais zco144 todos fert sql
todos_fert_sql = todos_fert_sql.merge(fert_zco144, on='Material', how='outer', suffixes=['', '_'], indicator=True)
#filtro apenas com materiais faltantes
todos_fert_sql = todos_fert_sql.loc[(todos_fert_sql['_merge'] == 'left_only')]
#deixando apenas coluna material
todos_fert_sql = todos_fert_sql[['Material']]
#colocando lista de materiais faltantes debaixo do dataframe dados
dados = pd.concat([dados, todos_fert_sql])

In [ ]:
#Copia todos códifo fert com unidade de medida basica
todos_fert_sql_Un = SQVI_BLOQUEI[['Material','UMB']]

#Merge dados com Unidade de medida basica
dados = dados.merge(todos_fert_sql_Un, on='Material',how='left')

In [ ]:
#Agrupando estoques
Estoque_Sql = Estoque_Sql.groupby(['Material'])['Estoq'].sum().reset_index().round(3)
#Merge dados com Estoque Material
dados = dados.merge(Estoque_Sql.drop_duplicates('Material'),how='left',on='Material')
#Merge dados com estoque componente
dados = (pd.merge(dados,Estoque_Sql , left_on='Comp',  right_on='Material', how='left').drop('Material_y',axis=1))

dados = dados[['Material_x','UMB','Estoq_x','Comp','UN','Estoq_y','Tipo']]
dados = dados.rename(columns={'Material_x': 'Material', 'Estoq_x':'Estoq', 'Estoq_y':'Estoq Comp'})

#padrão do material
dados  = dados.merge(Padrão_de_material, on='Material', how='left')
#frequencia
dados  = dados.merge(sqvi_zpp89_frequencia_venda, on='Material', how='left')


In [ ]:
#dados['Quant. CX Estoq'] = dados.Estoq / dados.Padrão

In [ ]:
dados  = dados.merge(ZSD138_CARTE, on='Material', how='left')
dados  = dados.merge(ZPP101, on='Material', how='left')

dados = (pd.merge(dados,ME2M , left_on='Material',  right_on='Material', how='left'))
dados = dados.rename(columns={'fornecer': 'fornecer Fert','UPP':'Upp'})

dados = (pd.merge(dados,ME2M , left_on='Comp',  right_on='Material', how='left')).drop('Material_y', axis=1)
dados = dados.rename(columns={'fornecer': 'fornecer Comp','UPP':'Upp','Material_x':'Material'})


dados  = dados.merge(ULTIMA_VENDA, on='Material', how='left')
dados  = dados.merge(ZMM34_VENDA, on='Material', how='left')
SQVI_BLOQUEI.drop(['UMB'],axis=1,inplace=True)
dados  = dados.merge(SQVI_BLOQUEI, on='Material', how='left')

dados = dados.fillna({'Média':0,'Venda 12 Meses':0,'Meses Ult Venda':9999,'Meses Cadastrado':0,'Frequencia':0,'Ordem Aberta':"Não",'Cart Venda':0})

In [ ]:
dados = dados.loc[(dados['GE'] != 'E') & (dados['Material'] != '9000SP') & (dados['Material'] != '9050SP')  & (dados['Material'] != '9100SP')  & (dados['Material'] != '9150SP') & (dados['Material'] != '9400SP') & (dados['Material'] != '9901SU') ]

#Ajustando descrições das colunas
dados = dados.rename(columns={'Desc': 'Desc GE', 'Desc.1':'Desc bloq prod', 'Desc.2':'Desc bloq venda', 'Desc.3':'Desc produto acabado'})

dados['Meses Ult Venda'] = dados['Meses Ult Venda'].apply(lambda x: "Nunca houve venda" if x == 9999 else x) 

In [ ]:
#ajustando descrição unidade de saida
dados['Unidade de Saida'] = dados['Unidade de Saida'].apply(lambda x: "CX" if x == "KI" else x)
dados['Unidade de Saida'] = dados['Unidade de Saida'].apply(lambda x: "SAC" if x == "BAG" else x)
dados['Unidade de Saida'] = dados['Unidade de Saida'].apply(lambda x: "PAC" if x == "PAK" else x)
dados['Unidade de Saida'] = dados['Unidade de Saida'].apply(lambda x: "GLL" if x == "GAL" else x)

#Atualizando nomes de TH para MIL de PAK para PAC
dados['UMB'] = dados['UMB'].apply(lambda x: "MIL" if x == "TH" else x)
dados['UMB'] = dados['UMB'].apply(lambda x: "PAC" if x == "PAK" else x)
dados['UMB'] = dados['UMB'].apply(lambda x: "PEC" if x == "ST" else x)
dados['UMB'] = dados['UMB'].apply(lambda x: "PAR" if x == "PAA" else x)

In [ ]:
name_excel = f'{(datetime.date.today()).year}-{(datetime.date.today()).month}-{(datetime.date.today()).day}'
dados.to_excel(f'U:\\Controladoria\\Cadastro\\14 - AUTOMATIZAÇÃO PYTHON\\Projetos\\Rafa pilot\\Marcar item para eliminar\\Pro\\Jupyter\\{name_excel} - Analise Final DD.xlsx',index=None, freeze_panes= (1,1))